In [1]:
import os
import re
from glob import glob
import json
import random

import pandas as pd
import numpy as np

from tqdm import tqdm

from gensim.corpora import Dictionary
from gensim.parsing.preprocessing import strip_punctuation, strip_multiple_whitespaces, strip_tags, strip_non_alphanum

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def read_corpus(folder):
    files = [y for x in os.walk(folder) for y in glob(os.path.join(x[0], '*.json'))]
    articles = []
    
    # attrs: id, contents
    for filepath in tqdm(files):
        # print(filepath)
        with open(filepath, 'r') as f:
            try:
                text = json.load(f)
            except:
                print('errors', filepath)
                continue
            
            for a in text['data']:
                article = {'id': a['id'], 'date': a['datePublished']}
                body = ""
                title = ""
                subjects = []
                
                for c in a['content']:
                    if c['type'] == 'heading':
                        title = c['text']
                    elif c['type'] == 'text':
                        body += " " + c['text']
                
                for s in a.get('subjects', []):
                    if s['title']['fi']:
                        subjects.append(s['title']['fi'].lower())
                    
                article['title'] = title
                article['body'] = body
                article['subjects'] = subjects
                
                articles.append(article)
        
    return articles

In [3]:
articles = read_corpus('data/yle-src/data/fi')

100%|██████████| 753/753 [05:25<00:00,  2.31it/s]


In [4]:
len(articles)

703673

In [5]:
articles[32]

{'id': '3-6450519',
 'date': '2013-01-14T17:00:23+0200',
 'title': 'Ruben Stiller: Kantasuomalainen, uussuomalainen, perussuomalainen vai pseudosuomalainen?',
 'body': ' Vai olenko sittenkin megasuomalainen?  Vaikeaa tämä suomalaisuus. Alussa oli maahanmuuttaja. Sitten hänet päätettiin nimetä _uussuomalaiseksi,_ jotta kaikki tajuaisivat, että maahanmuuttajakin voi olla suomalainen. Tämä ei kuitenkaan riittänyt: jotenkin se maahanmuuttaja oli erotettava varsinaisista suomalaisista, joten joukkoomme syntyi yhtäkkiä valtava määrä ns. _kantasuomalaisia_. En ymmärrä tämän kielipelin mielekkyyttä - miksi maahanmuuttaja muutetaan uussuomalaiseksi, jos tarkoituksena on kuitenkin pitää yllä sitä erottelua, josta yritettiin luopua? Miksi maahanmuuttaja-sanalle\xa0täytyy keksiä poliittisesti korrekti vastine, joka vain korostaa sitä, kuinka suomalaisia sitä ollaan? Miten kauan uussuomalaisen pitää olla uussuomalainen ennen kuin hän muuttuu kantasuomalaiseksi? Koska suomalaisuus on nykyään valtava

In [ ]:
with open('data/yle-articles.json', 'w', encoding='utf-8') as f:
    json.dump(articles, f, ensure_ascii=False, indent=4)

In [2]:
def load_data(filepath):
    with open(filepath, 'r') as f:
        articles = json.load(f)
        
    return articles

In [3]:
# load data
articles = load_data('data/yle-articles.json')

In [4]:
def create_samples(articles, output, k=1000):
    samples = random.sample(articles, k)
    with open(output, 'w', encoding='utf-8') as f:
        json.dump(samples, f, ensure_ascii=False, indent=4)
    
    return samples

In [5]:
# just wrap some samples to quick test what is going on, no need for production
samples = create_samples(articles, 'data/yle-articles-dev.json', k=20000)

In [6]:
samples[1]

{'id': '3-8010078',
 'date': '2015-05-22T11:46:37+0300',
 'title': 'Kemppainen valitsi voittamisen - "Puhutaan NHL:n huippuorganisaatiosta"',
 'body': ' Joonas Kemppainen valitsi Boston Bruinsin mahdollisuudekseen tehdä läpimurto NHL:ään. Sentteri korostaa työnteon merkitystä. Joonas Kemppaisella on edessään hikinen kesä, sanoivat sääennusteet mitä tahansa. Boston Bruinsin kanssa yhden vuoden kaksisuuntaisen sopimuksen tehnyt hyökkääjä tietää, että sopimus ei vielä tarkoita pelipaikkaa.  \\- Ihan hyvältä tuntuu, mutta paljon on töitä edessä kesällä. Lähdetään sillä asenteella, että saadaan se paikka NHL:stä, Kemppainen vakuuttaa. \\- Eihän se sopimus vielä mitään takaa, mutta hienoa, että noin hyvän mahdollisuuden sain. Viime kevät päättyi menestykseen tottuneen seuran kannalta vaisusti. Päävalmentaja Claude Julien ei saanut piiskattua miehistöään entisenlaiseen iskuun ja Bruins jäi rannalle pudotuspeleistä. Edellisen kerran itärannikon seura oli Stanley Cupin loppuottelussa 2013 ja to

In [5]:
def tokenize_articles(articles, max_len=30):
    
    for article in tqdm(articles):
        try:
            text = article['title'] + ' ' + article['body']
        except:
            text = article['body']
            print(article['title'])

        text = strip_multiple_whitespaces(text)
        # remove everything except words, spaces, digits and dash -
        # text = re.sub(r'[^\s\d\wäöüåÄÖÜÅ\-]', '', text)
        text = strip_tags(text)
        text = strip_non_alphanum(text)
        text = text.lower()
        tokens = text.split()
        tokens = [w for w in tokens if len(w) <= max_len]
        
        # do the same for the subjects
        subjects = []
        
        for s in article['subjects']:
            # s = re.sub(r'[^\s\d\wäöüåÄÖÜÅ\-]', '', s)
            if s is not None:
                s = strip_non_alphanum(s)
                s = s.lower()
                s_tokens = s.split()
                subjects += s_tokens
        
        # empty body for saving space
        article['tokens'] = tokens
        article['subjects'] = subjects
        article['body'] = ''
    
    return articles

In [5]:
articles[0]

{'id': '3-6567618',
 'date': '2013-04-05T23:05:35+0300',
 'title': 'FoKoPon putoaminen sai sinetin',
 'body': ' Forssan Koripoikien visiitti Korisliigassa jäi lyhyeksi, sillä sarjanousijan kohtalona on paluu ykkösdivisioonaan. Asia\nvarmistui perjantaina Tampereella, kun FoKoPo hävisi Pyrinnölle. Pyrinnön\nvoittolukemat olivat 113-78. Voittokaan ei olisi pelastanut FoKoPoa, sillä\nSalon Vilpas löi Uudenkaupungin Korihait 67-59. FoKoPo\njaksoi taistella Tampereella ensimmäisen jakson tasaisesti (17-17). Sitten\nrepesi. Pyrintö vei toisen kympin rökäleluvuin, johtaen tauolla 48-30.\nLoppulukemat olivat 113-78. Tamperelaisten \nyllättäjä oli Osku Heinonen. Heinonen on Pyrinnön oma kasvatti. Peliaikaa\nkertyi 22 minuuttia ja pisteitä 24. Carl Lindbom (29/11) ja Lamonte Ulmer (19)\nolivat myös pelivireessä. FoKoPon\nparhaat olivat Andrais Thornton (30) ja Sami Ikävalko (21). Ensi\ntorstaina FoKoPo päättää Korisliigakauden Feeniksissä, kohtaamalla lo 18.30\nalkavassa ottelussa Salon Vilppaan

In [6]:
articles = tokenize_articles(articles)

  5%|▍         | 33006/702673 [00:07<02:39, 4199.49it/s]

['resource', {'id': '6385775'}]


100%|██████████| 702673/702673 [03:14<00:00, 3618.74it/s]


In [7]:
articles[89]

{'id': '3-6475125',
 'date': '2013-01-31T16:16:04+0200',
 'title': 'AHL-seuran tempaus: jää värjätään pinkiksi',
 'body': '',
 'subjects': ['urheilu',
  'jääkiekko',
  'worcester',
  'sharks',
  'american',
  'hockey',
  'league',
  'rintasyöpä',
  'pohjois',
  'amerikan',
  'jääkiekko',
  'providence',
  'bruins',
  'harri',
  'säteri',
  'naisten',
  'syövät',
  'rinnat',
  'talviurheilu',
  'miesten',
  'jääkiekko',
  'rintakehä',
  'joukkueurheilu',
  'hyväntekeväisyys'],
 'tokens': ['ahl',
  'seuran',
  'tempaus',
  'jää',
  'värjätään',
  'pinkiksi',
  'worcester',
  'sharksin',
  'kotipelissä',
  'kerätään',
  'rahaa',
  'rintasyövän',
  'tutkimusta',
  'varten',
  'jääkiekon',
  'ahl',
  'liigassa',
  'pelaava',
  'worcester',
  'sharks',
  'pelaa',
  'lauantain',
  'kotiottelunsa',
  'poikkeuksellisissa',
  'olosuhteissa',
  'sharksin',
  'kotikaukalon',
  'jää',
  'värjätään',
  'pinkiksi',
  'kyseessä',
  'on',
  'hyväntekeväisyystempaus',
  'rintasyövän',
  'tutkimuksen',
 

In [8]:
with open('data/yle-tokenized.json', 'w', encoding='utf-8') as f:
    json.dump(articles, f, ensure_ascii=False)

In [13]:
# this is not neccessary for production, just get some sample to see what is going on.
samples = tokenize_articles(samples)

100%|██████████| 20000/20000 [00:01<00:00, 15135.42it/s]


In [14]:
with open('data/yle-tokenized-dev.json', 'w', encoding='utf-8') as f:
    json.dump(samples, f, ensure_ascii=False)